Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [29]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
#from scipy import stats
#from scipy import special

In [30]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [31]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [32]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    cursor.execute("EXEC [dbo].[sp_get_precios_recolectados_mes] 2024, 2")
    # Initialize an empty list to store rows
    rows = []
    # Iterate over the cursor and append each row to the list as a dictionary
    for row in cursor.fetchall():
        rows.append(dict(zip([column[0] for column in cursor.description], row)))
    # Create DataFrame from the list of dictionaries
    boletas = pd.DataFrame(rows)
    
    cursor.execute("EXEC sp_get_indice_grupo 2024, 2")
    # Initialize an empty list to store rows
    rows = []
    # Iterate over the cursor and append each row to the list as a dictionary
    for row in cursor.fetchall():
        rows.append(dict(zip([column[0] for column in cursor.description], row)))
    ponderaciones = pd.DataFrame(rows)

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [33]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [41]:
# Filter rows based on conditions (commented out as it seems these filters are optional)
# boletas = boletas[boletas['tp_poll_impute_status'].isin([351821, 351824])]
# boletas = boletas[boletas['detail_tp_poll_status'].isin([351634, 351635])]

# Selecciona las columnas que se van a utilizar
boletas = boletas[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']]
boletas.insert(1, 'cod_prod', 
               np.where(boletas['codigo_articulo'].str.len() > 8, 
                        boletas['codigo_articulo'].str.slice(0, -2), 
                        boletas['codigo_articulo'].str.slice(0, -1)))
boletas['cod_prod'] = boletas['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
boletas['cantidad_anterior'] = boletas.apply(lambda row: row['cantidad_actual'] if row['cantidad_anterior'] == 0 or pd.isnull(row['cantidad_anterior']) else row['cantidad_anterior'], axis=1)
boletas['precio_anterior'] = boletas.apply(lambda row: row['precio_actual'] if row['precio_anterior'] == 0 or pd.isnull(row['precio_anterior']) else row['precio_anterior'], axis=1)

In [42]:
boletas

,region,cod_prod,codigo_articulo,articulo,cantidad_anterior,cantidad_actual,precio_anterior,precio_actual,nt_tipo
0,1,0540301,5403012,Olla y cacerola de 1 a 2lt,1.00,1.00,145.00,145.00,NaN
1,1,0312306,3123061,Vestidos formales con mangas de largo hasta la...,1.00,1.00,250.00,250.00,NaN
2,1,0721101,7211011,Llantas para automóvil 185/65 R14,1.00,1.00,400.00,400.00,NaN
3,1,0411101,4111013,Alquiler de apartamento (con o sin servicios),1.00,1.00,1700.00,1700.00,NaN
4,1,0722401,7224013,Aceite de motor sintético,946.00,946.00,70.00,70.00,NaN
...,...,...,...,...,...,...,...,...,...
55084,8,0117402,1174021,Rábanos frescos,661.00,661.00,7.00,7.00,NaN
55085,8,0117403,1174031,Cebollas blancas,454.00,454.00,10.00,10.00,NaN
55086,8,0117404,1174041,Maíz verde fresco,844.00,844.00,7.00,7.00,NaN
55087,8,0117405,1174051,Verduras frescas o refrigeradas limpias y cort...,1.00,1.00,1.00,1.00,NaN
